In [73]:
from glob import glob
import numpy as np
import pandas as pd
import pprint
from IPython.display import display, HTML

In [58]:
# Table for each TSP algorithm
# - for each algo, report 
#   1) time, 
#   2) distance, 
#   3) relative error: (alg-opt)/opt
# - round time and relative error to 2 and 4 digits respectively
# - For local search, the values will be averages over all the runs for a particular instance

In [59]:
solutions = pd.read_csv("DATA/solutions.csv", index_col=0)
solutions

,Value
Instance,
Atlanta,2003763
Berlin,7542
Boston,893536
Champaign,52643
Cincinnati,277952
Denver,100431
NYC,1555060
Philadelphia,1395981
Roanoke,655454


In [60]:
solutions['Value']["Atlanta"]

2003763

In [83]:
algos = {
    "BnB": "bnbtraces", 
    "Approx": "approxout", 
    "LS1": "lsbatchout", 
    "LS2": "lsbatchout"
}

def generate_table(algo: str, root_dir="traces"):
    
    parent_dir = f"{root_dir}/{algos[algo]}"
    
    city_data = {}
    
    for city in solutions.index:
        
        times = []
        distances = []
        
        pattern = f"{parent_dir}/{city}{algo}/*.trace"
        files = glob(pattern)
        
        for file in files:
            trace_data = pd.read_csv(
                file,
                header=None,
                names=['time', 'distance']
            )
            cur_distances = list(trace_data['distance'])
            cur_times = list(trace_data['time'])
            
            # Find first time to get best distance
            best_distance = cur_distances[-1]
            for i, d in enumerate(cur_distances):
                if d == best_distance:
                    break
            best_time = cur_times[i]
            
            times.append(best_time)
            distances.append(best_distance)
            
        times = np.array(times)
        distances = np.array(distances)
        
        opt_dist = solutions['Value'][city]
        rel_error = (distances - opt_dist) / opt_dist
        
        city_data[city] = {
            'Time (s)': np.around(times.mean(),2),
            'Sol.Qual.': int(distances.mean()),
            'RelErr': np.around(rel_error.mean(), 4)
        }
            
    return city_data    
    
all_data = {}
for algo in algos:
    all_data[algo] = pd.DataFrame(
        data=generate_table(algo)
    ).T
    print(algo)
    display(all_data[algo])

BnB


,Time (s),Sol.Qual.,RelErr
Atlanta,17.58,2003763.0,0.0000
Berlin,69.09,8892.0,0.1790
Boston,330.81,945712.0,0.0584
Champaign,571.51,53593.0,0.0180
Cincinnati,0.00,277952.0,0.0000
Denver,78.90,115531.0,0.1504
NYC,97.94,1726479.0,0.1102
Philadelphia,1.90,1421930.0,0.0186
Roanoke,516.00,747999.0,0.1412
SanFrancisco,0.00,897275.0,0.1075


Approx


,Time (s),Sol.Qual.,RelErr
Atlanta,0.00,2574256.0,0.2847
Berlin,0.00,9867.0,0.3083
Boston,0.00,1082076.0,0.2110
Champaign,0.00,67662.0,0.2853
Cincinnati,0.00,308517.0,0.1100
Denver,0.01,127523.0,0.2698
NYC,0.00,1839455.0,0.1829
Philadelphia,0.00,1710379.0,0.2252
Roanoke,0.05,800949.0,0.2220
SanFrancisco,0.01,1120431.0,0.3829


LS1


,Time (s),Sol.Qual.,RelErr
Atlanta,0.39,2003763.0,0.0000
Berlin,5.45,8355.0,0.1078
Boston,4.46,924436.0,0.0346
Champaign,5.70,56914.0,0.0811
Cincinnati,0.04,277952.0,0.0000
Denver,5.80,117427.0,0.1692
NYC,5.66,1695038.0,0.0900
Philadelphia,2.09,1395981.0,0.0000
Roanoke,6.55,1097548.0,0.6745
SanFrancisco,6.10,1056455.0,0.3040


LS2


,Time (s),Sol.Qual.,RelErr
Atlanta,0.09,2003763.0,0.0000
Berlin,10.38,7579.0,0.0050
Boston,3.28,893838.0,0.0003
Champaign,17.22,52730.0,0.0017
Cincinnati,0.05,277952.0,0.0000
Denver,17.42,101716.0,0.0128
NYC,11.95,1563495.0,0.0054
Philadelphia,0.37,1395981.0,0.0000
Roanoke,22.55,672828.0,0.0265
SanFrancisco,12.50,819772.0,0.0118


In [85]:
latex_data = {}
for algo in all_data:
    print(algo)
    latex_data[algo] = all_data[algo].to_latex()
    print(latex_data[algo])

BnB
\begin{tabular}{lrrr}
\toprule
{} &  Time (s) &  Sol.Qual. &  RelErr \\
\midrule
Atlanta      &     17.58 &  2003763.0 &  0.0000 \\
Berlin       &     69.09 &     8892.0 &  0.1790 \\
Boston       &    330.81 &   945712.0 &  0.0584 \\
Champaign    &    571.51 &    53593.0 &  0.0180 \\
Cincinnati   &      0.00 &   277952.0 &  0.0000 \\
Denver       &     78.90 &   115531.0 &  0.1504 \\
NYC          &     97.94 &  1726479.0 &  0.1102 \\
Philadelphia &      1.90 &  1421930.0 &  0.0186 \\
Roanoke      &    516.00 &   747999.0 &  0.1412 \\
SanFrancisco &      0.00 &   897275.0 &  0.1075 \\
Toronto      &      7.29 &  1239724.0 &  0.0541 \\
UKansasState &      0.00 &    62962.0 &  0.0000 \\
UMissouri    &      3.75 &   156251.0 &  0.1774 \\
\bottomrule
\end{tabular}

Approx
\begin{tabular}{lrrr}
\toprule
{} &  Time (s) &  Sol.Qual. &  RelErr \\
\midrule
Atlanta      &      0.00 &  2574256.0 &  0.2847 \\
Berlin       &      0.00 &     9867.0 &  0.3083 \\
Boston       &      0.00 &  1082076